In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI
from dotenv import load_dotenv
import os
from pathlib import Path
import pickle

In [2]:
load_dotenv()

False

In [3]:
documentDirectory = input("Input a directory to be scanned: ")
formattedPath = Path(documentDirectory)
formattedPath
documents = SimpleDirectoryReader(formattedPath).load_data()
print(documents[0])

Input a directory to be scanned:  Data


Doc ID: dc7140c6-5a33-4945-8481-62d3536f2f59
Text: The Union Budget 2024, presented by Finance Minister Nirmala
Sitharaman, has set the stage for a transformative period in India's
economic landscape. With a focus on education, employment, urban
development, and technological advancement, the budget aims to propel
India towards sustainable growth and prosperity.  The Union Budget
2024, presented...


In [4]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=250, chunk_overlap=10)
nodes = text_splitter.get_nodes_from_documents(documents=documents)

In [16]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding

chroma_client = chromadb.EphemeralClient()
collection_name = 'BudgetArticle1'
if collection_name in [coll.name for coll in chroma_client.list_collections()]:
        chroma_client.delete_collection(collection_name)
chroma_collection = chroma_client.create_collection(collection_name)
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [13]:
index = VectorStoreIndex(nodes=nodes, storage_context=storage_context, embed_model=OpenAIEmbedding())
#full_path = os.path.join(formattedPath, documentDirectory+'_index.pkl')
#with open(full_path, 'wb') as file:
#    pickle.dump(index, file)

#with open(full_path, 'rb') as file:
#    index = pickle.load(file)
#    print(index)

Insert of existing embedding ID: 0fc845d8-b888-478b-b33d-105c52830d3f
Insert of existing embedding ID: ddaedf2b-cad0-48c8-8fa4-c055110105fe
Insert of existing embedding ID: e9e30c35-bafe-4ce2-b675-c3f39abfd694
Insert of existing embedding ID: 9b548a18-8721-4d98-955f-afd8c95c60dd
Insert of existing embedding ID: c5c0443d-3fc7-4fbe-8473-ca14c0fac5cc
Insert of existing embedding ID: 29cfadd5-7017-48ad-9e3a-e2579aa2809f
Insert of existing embedding ID: 5a752e2e-c286-4895-8fa1-80855cefcb95
Insert of existing embedding ID: 7310cc76-7b54-4f2d-897f-71e496c757f4
Insert of existing embedding ID: f77d442a-b4a9-4c7f-abba-fe091e00890d
Insert of existing embedding ID: 85ef20b2-5fc5-45ce-bd36-e22080e47607
Insert of existing embedding ID: a4570531-ed10-4d48-9f7c-6b85fc2316c7
Insert of existing embedding ID: e4f40c53-c2c3-4bd3-b380-62ca5620f407
Insert of existing embedding ID: 19e5eb74-58eb-4df0-ba4f-e42cc2e962ce
Insert of existing embedding ID: 44dc3bb9-04c5-4b72-a12b-1f611591d1df
Insert of existing e

In [14]:
#question = "Explain the article in brief"
question = input("Enter your question: ")
retriever = index.as_retriever()
retriever.retrieve(question)

Enter your question:  How is budget


AttributeError: 'ChromaVectorStore' object has no attribute '_collection'

In [8]:
llm = OpenAI(model="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query(question)
print(response)

I'm unable to provide information on the weather as it is not mentioned in the context provided.


In [9]:
#This helps in understanding the prompts which query engine uses to give answers
prompts = query_engine.get_prompts()
print(prompts)

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x000001C2ABCCC680>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageR